In [21]:
import sys
sys.path.append('../')

In [22]:
import AbreivBot.Functions as functions
import AbreivBot.Models.BaseModels as base_model

In [23]:
import pandas as pd
import numpy as np
import string

In [24]:
import tensorflow as tf
devices = tf.config.list_physical_devices()

In [25]:
devices

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [26]:
abbrev = pd.read_csv('TrainAndTest/traindata/cleaned_abbreviations.csv')
abbrev = abbrev[abbrev.columns[1:]]

In [27]:
abbrev['parse_ylabel'] = abbrev.apply(lambda x: ''.join(functions.encode_ylabel(x['Abbreviation'], x['Word'])), axis=1)
abbrev['WordLength'] = abbrev['Word'].apply(lambda x: len(x))

In [28]:
abbrev.head()

,Abbreviation,Word,parse_ylabel,WordLength
0,A.D.,Anno Domini,A.@@@D.@@@@,11
1,A.V.,Authorized Version,A.@@@@@@@@@V.@@@@@,18
2,Abbrev.,abbreviation,Abbrev.@@@@@,12
3,Abbrev.,abbreviations,Abbrev.@@@@@@,13
4,Abd.,Aberdeen,Ab@@d.@@,8


In [29]:
possible_chars = string.ascii_letters + string.digits + ' .@'
possible_chars

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .@'

In [30]:
model = base_model.BaseModel(possible_chars)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, None, len(possible_chars)))

2023-04-15 05:17:30.022433: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-15 05:17:30.023889: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-15 05:17:30.025321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [31]:
train = abbrev.sample(frac=0.8, random_state=1337)
test = abbrev.drop(train.index)

In [32]:
import AbreivBot.TrainModel as train_model
import AbreivBot.TestModel as test_model

In [33]:
history = train_model.train(model, train, possible_chars, verbose=0)

2023-04-15 05:17:30.832489: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-15 05:17:30.834589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-15 05:17:30.836335: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [34]:
model.summary()

Model: "base_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  65        
                                                                 
 flatten_1 (Flatten)         multiple                  0 (unused)
                                                                 
 lstm_1 (LSTM)               multiple                  34060     
                                                                 
 dense_1 (Dense)             multiple                  4290      
                                                                 
Total params: 38,415
Trainable params: 38,415
Non-trainable params: 0
_________________________________________________________________


In [35]:
results = test_model.test(model, test, possible_chars, verbose=0)

2023-04-15 05:18:42.829174: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,8,65]
	 [[{{node Placeholder/_1}}]]
2023-04-15 05:18:43.009079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [21,13,65]
	 [[{{node Placeholder/_1}}]]
2023-04-15 05:18:43.097647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [

In [36]:
results

{8: [1.8503479957580566, 0.4921875],
 13: [1.8934828042984009, 0.5421245694160461],
 10: [1.7252575159072876, 0.5367347002029419],
 11: [2.1646482944488525, 0.4890282154083252],
 14: [2.363931894302368, 0.5],
 9: [2.145416736602783, 0.4444444477558136],
 12: [2.1820812225341797, 0.5053763389587402],
 15: [2.238980531692505, 0.4833333194255829],
 7: [2.075833320617676, 0.45812806487083435],
 16: [2.485269546508789, 0.4375],
 5: [2.5739824771881104, 0.5555555820465088],
 20: [2.56856107711792, 0.550000011920929],
 19: [2.4411230087280273, 0.5263158082962036],
 6: [2.0548317432403564, 0.46296295523643494],
 4: [2.175161838531494, 0.625],
 22: [1.4844985008239746, 0.6818181872367859],
 18: [2.8690292835235596, 0.35185185074806213],
 17: [2.206827163696289, 0.47058823704719543],
 21: [1.9046298265457153, 0.6666666865348816]}

In [38]:
model.save_weights('Models/TrainedModels/BaseModel')